In [1]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir

    path.append(dir(path[0]))
    __package__ = "dash_app"

In [2]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map, map_columns
from settings import *

In [3]:
data_path = DATA_PATH+'Oceanic_Domain/3.9OceanColour/Figure3.20/'
data_csv = data_path + 'Figure3.20_data.csv'

In [4]:
"""
Tidy data for charts
"""
columns_dict = {'datetime':'(month)',
               'mean__monthly__mass_concentration_of_chlorophyll_a_in_sea_water':'Monthly mean (46º-58º N, 5º-26º W)',
                  }
xls = pd.ExcelFile(
    data_path+'Monthly_Mean_ChlorophyllA_Concentration_IrishMaritimeBoundaryContinentalShelf.xlsx')
original_df = pd.read_excel(xls, 'SWFMO_CHLO.CR.l3m_data.Oct1997-',skiprows=8)
df = map_columns(columns_dict, original_df)
df['datetime'] = pd.to_datetime(df['datetime'],format='%Y-%m-%d %H:%M:%S')
#df.to_csv(data_csv) #commented out as the file should be kept as is

In [5]:
df = pd.read_csv(data_csv, index_col=0)
df['datetime'] = pd.to_datetime(df['datetime'])
df['Mean'] = df['mean__monthly__mass_concentration_of_chlorophyll_a_in_sea_water']

In [6]:
break_points = [
        df['Mean'].min(),
        np.percentile(df['Mean'], 5),
        np.percentile(df['Mean'], 25),
        np.percentile(df['Mean'], 50),
        np.percentile(df['Mean'], 75),
        np.percentile(df['Mean'], 95)
]
break_points

[0.564792454,
 0.6019876118999999,
 0.658065066,
 0.725695461,
 0.81297628575,
 0.92176884695]

In [7]:
df['Mean']=df['mean__monthly__mass_concentration_of_chlorophyll_a_in_sea_water']
numTotal=len(df)
numPurple=len(df[(df['Mean']<0.56)])
numBlue=len(df[(df['Mean']>=0.56)&(df['Mean']<0.6)])
numGreen=len(df[(df['Mean']>=0.66)&(df['Mean']<0.73)])
numYellow=len(df[(df['Mean']>=0.73)&(df['Mean']<0.81)])
numOrange=len(df[(df['Mean']>=0.81)&(df['Mean']<0.92)])
numRed=len(df[(df['Mean']>=0.92)])
perPurple=numPurple/numTotal
perBlue=numBlue/numTotal
perGreen=numGreen/numTotal
perYellow=numYellow/numTotal
perOrange=numOrange/numTotal
perRed=numRed/numTotal
print([perPurple,
       perBlue, 
       perGreen,
       perYellow,
       perOrange,
       perRed])
print([perPurple,
       perPurple+perBlue, 
       perPurple+perBlue+perGreen,
       perPurple+perBlue+perGreen+perYellow,
       perPurple+perBlue+perGreen+perYellow+perOrange,
       perPurple+perBlue+perGreen+perYellow+perOrange+perRed])

[0.0, 0.045454545454545456, 0.25252525252525254, 0.2222222222222222, 0.21212121212121213, 0.05555555555555555]
[0.0, 0.045454545454545456, 0.297979797979798, 0.5202020202020202, 0.7323232323232324, 0.787878787878788]


In [8]:
def generateDiscreteColourScale(colour_set):
    #colour set is a list of lists
    colour_output = []
    num_colours = len(colour_set)
    divisions = 1./num_colours
    c_index = 0.    # Loop over the colour set
    for cset in colour_set:
        num_subs = len(cset)
        sub_divisions = divisions/num_subs        # Loop over the sub colours in this set
        for subcset in cset:
            colour_output.append((c_index,subcset))
            colour_output.append((c_index + sub_divisions-
                .001,subcset))
            c_index = c_index + sub_divisions
    colour_output[-1]=(1,colour_output[-1][1])
    return colour_output

color_schemes = ["rgb(98, 55, 155)",
                 "rgb(184, 197, 229)",
                 "rgb(166, 206, 93)",
                 "rgb(255, 254, 66)",
                 "rgb(239, 191, 49)",
                 "rgb(219, 32, 1)"
                ]
colorscale = generateDiscreteColourScale(color_schemes)

In [12]:
colorscale=[
        # 5% are to be purple
        [0.0, "rgb(98, 55, 155)"],
        [0.565, "rgb(98, 55, 155)"],
        # 20% are to be blue
        [0.565, "rgb(184, 197, 229)"],
        [0.602, "rgb(184, 197, 229)"],
        # 25% are to be green
        [0.602, "rgb(166, 206, 93)"],
        [0.658, "rgb(166, 206, 93)"],
        # 25% are to be yellow
        [0.658, "rgb(255, 254, 66)"],
        [0.726, "rgb(255, 254, 66)"],
        # 20% are to be orange
        [0.726, "rgb(239, 191, 49)"],
        [0.813, "rgb(239, 191, 49)"],
        # 5% are to be red
        [0.813, "rgb(219, 32, 1)"],
        [1, "rgb(219, 32, 1)"],
    ]

# colorscale=[
#         # 5% are to be purple
#         [0, "rgb(98, 55, 155)"],
#         [np.percentile(df['Mean'], 5), "rgb(98, 55, 155)"],
#         # 20% are to be blue
#         [np.percentile(df['Mean'], 5), "rgb(184, 197, 229)"],
#         [np.percentile(df['Mean'], 25), "rgb(184, 197, 229)"],
#         # 25% are to be green
#         [np.percentile(df['Mean'], 25), "rgb(166, 206, 93)"],
#         [np.percentile(df['Mean'], 50), "rgb(166, 206, 93)"],
#         # 25% are to be yellow
#         [np.percentile(df['Mean'], 50), "rgb(255, 254, 66)"],
#         [np.percentile(df['Mean'], 75), "rgb(255, 254, 66)"],
#         # 20% are to be orange
#         [np.percentile(df['Mean'], 75), "rgb(239, 191, 49)"],
#         [np.percentile(df['Mean'], 95), "rgb(239, 191, 49)"],
#         # 5% are to be red
#         [np.percentile(df['Mean'], 95), "rgb(219, 32, 1)"],
#         [1, "rgb(219, 32, 1)"],
#     ]
# colorscale=[
#             [0, "rgb(98, 55, 155)"],
#         # 5% are to be purple
#         [np.percentile(df['Mean'], 5), "rgb(98, 55, 155)"],
#         # 20% are to be blue
#         [np.percentile(df['Mean'], 25), "rgb(184, 197, 229)"],
#         # 25% are to be green
#         [np.percentile(df['Mean'], 50), "rgb(166, 206, 93)"],
#         # 25% are to be yellow
#         [np.percentile(df['Mean'], 75), "rgb(255, 254, 66)"],
#         # 20% are to be orange
#         [np.percentile(df['Mean'], 95), "rgb(239, 191, 49)"],
#         # 5% are to be red
#         [1, "rgb(219, 32, 1)"],
#     ]
colorscale=[
        # 5% are to be purple
        [0, "rgb(98, 55, 155)"],
        [0.085, "rgb(98, 55, 155)"],
        # 20% are to be blue
        [0.085, "rgb(184, 197, 229)"],
        [0.30, "rgb(184, 197, 229)"],
        # 25% are to be green
        [0.30, "rgb(166, 206, 93)"],
        [0.5, "rgb(166, 206, 93)"],
        # 25% are to be yellow
        [0.5, "rgb(255, 254, 66)"],
        [0.75, "rgb(255, 254, 66)"],
        # 20% are to be orange
        [0.75, "rgb(239, 191, 49)"],
        [0.92, "rgb(239, 191, 49)"],
        # 5% are to be red
        [0.92, "rgb(219, 32, 1)"],
        [1, "rgb(219, 32, 1)"]
]

colorscale = generateDiscreteColourScale(color_schemes)

trace = go.Heatmap(
    z=df['mean__monthly__mass_concentration_of_chlorophyll_a_in_sea_water'],
    x=df['datetime'].dt.month,
    y=df['datetime'].dt.year,
    text=df['datetime'],
    colorscale='Rainbow',
    hovertemplate='%{text|%b %Y}<br>'+
    'Concentration: %{z:.2f} mg/m\u00B3<extra></extra>'
)


In [29]:
figure_3_20 = go.Figure(data=trace, layout=TIMESERIES_LAYOUT)
figure_3_20

In [30]:

figure_3_20.update_layout(
    yaxis=dict(
        title='Year',
        nticks=12),
    xaxis=dict(
        title='Month',
        ticktext=['Jan','Feb','Mar','Apr','May','June','July','Aug','Sep','Oct','Nov','Dec'],
        showgrid=False,
       tickvals=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    ))
    
figure_3_20